In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from Utils.setupDataset import get_dataset, load
from Utils.feature_vector import get_feature_names
import numpy as np
from Utils.univariate_featureSelection import get_selectedFeatures, get_metrics, metrics_feature_selection

def metrics_feature_selection(ranks, X_train, y_train, X_test, y_test, n):

calls get_metrics

In [ ]:
def featureSelectionResults(trainSet, validationSet, rankfile='Datasets/rank_selections.cvs', RFE_rankfile='Datasets/RFE_ranks.csv', n=25):
    # 1. get the feature rankings
    # 1.1 univariate ranks from file
    ranks = getTopFeatures(trainSet, rankfile)
    
    # 1.2 compute RFE ranks
    RFE_ranks = getRFE_ranking(trainset, RFE_rankfile)
    ranks = np.concatenate((ranks, RFE_ranks), axis=1)
    
    X_train = trainSet['data']
    y_train = trainSet['target'
    X_test = validationSet['data']
    y_test = validationSet['target']
       
    metrics = metrics_feature_selection(ranks, X_train, y_train, X_test, y_test, n)
    
    return SelectionMetrics(metrics, ranks, n)